In [1]:
import os
import openai
import json
import re
from IPython.display import display, HTML

#API Keys
os.environ['OPENAI_API_TOKEN'] = 'YOUR_API_KEY_HERE'

In [59]:
def strict_output(system_prompt, user_prompt, output_format, default_category = "", output_value_only = False,
                  model = 'gpt-3.5-turbo', temperature = 0, num_tries = 2, verbose = False):
    ''' Ensures that OpenAI will always adhere to the desired output json format. 
    Uses rule-based iterative feedback to ask GPT to self-correct.
    Keeps trying up to num_tries it it does not. Returns empty json if unable to after num_tries iterations.
    If output field is a list, will treat as a classification problem and output best classification category.
    Text enclosed within < > will generated by GPT accordingly'''

    # if the user input is in a list, we also process the output as a list of json
    list_input = isinstance(user_prompt, list)
    # if the output format contains dynamic elements of < or >, then add to the prompt to handle dynamic elements
    dynamic_elements = '<' in str(output_format)
    # if the output format contains list elements of [ or ], then we add to the prompt to handle lists
    list_output = '[' in str(output_format)
    
    # start off with no error message
    error_msg = ''
    
    for i in range(num_tries):
        
        output_format_prompt = f'''\nYou are to output the following in json format: {output_format}. 
Do not put quotation marks ' or " or escape character \ in the output fields.'''
        
        if list_output:
            output_format_prompt += f'''\nIf output field is a list, classify output into the best element of the list.'''
        
        # if output_format contains dynamic elements, process it accordingly
        if dynamic_elements: 
            output_format_prompt += f'''
Any text enclosed by < and > indicates you must generate content to replace it. Example input: Go to <location>, Example output: Go to the garden
Any output key containing < and > indicates you must generate the key name to replace it. Example input: {{'<location>': 'description of location'}}, Example output: {{school: a place for education}}'''

        # if input is in a list format, ask it to generate json in a list
        if list_input:
            output_format_prompt += '''\nGenerate a list of json, one json for each input element.'''
            
        # Use OpenAI to get a response
        response = openai.ChatCompletion.create(
          temperature = temperature,
          model=model,
          messages=[
            {"role": "system", "content": system_prompt + output_format_prompt + error_msg},
            {"role": "user", "content": str(user_prompt)}
          ]
        )

        res = response['choices'][0]['message']['content'].replace('\'', '"')
        
        # ensure that we don't replace away aprostophes in text 
        res = re.sub(r"(\w)\"(\w)", r"\1'\2", res)

        if verbose:
            print('System prompt:', system_prompt + output_format_prompt + error_msg)
            print('\nUser prompt:', str(user_prompt))
            print('\nGPT response:', res)
        
        # try-catch block to ensure output format is adhered to
        try:
            output = json.loads(res)
            if isinstance(user_prompt, list):
                if not isinstance(output, list): raise Exception("Output format not in a list of json")
            else:
                output = [output]
                
            # check for each element in the output_list, the format is correctly adhered to
            for index in range(len(output)):
                for key in output_format.keys():
                    # unable to ensure accuracy of dynamic output header, so skip it
                    if '<' in key or '>' in key: continue
                    # if output field missing, raise an error
                    if key not in output[index]: raise Exception(f"{key} not in json output")
                    # check that one of the choices given for the list of words is an unknown
                    if isinstance(output_format[key], list):
                        choices = output_format[key]
                        # ensure output is not a list
                        if isinstance(output[index][key], list):
                            output[index][key] = output[index][key][0]
                        # output the default category (if any) if GPT is unable to identify the category
                        if output[index][key] not in choices and default_category:
                            output[index][key] = default_category
                        # if the output is a description format, get only the label
                        if ':' in output[index][key]:
                            output[index][key] = output[index][key].split(':')[0]
                            
                # if we just want the values for the outputs
                if output_value_only:
                    output[index] = [value for value in output[index].values()]
                    # just output without the list if there is only one element
                    if len(output[index]) == 1:
                        output[index] = output[index][0]
                    
            return output if list_input else output[0]

        except Exception as e:
            error_msg = f"\n\nResult: {res}\n\nError message: {str(e)}"
            print("An exception occurred:", str(e))
            print("Current invalid json format:", res)
         
    return {}

In [144]:
def display_images(option1, option2, option3):
    defaulturl = "https://th.bing.com/th/id/OIG.lVXjWwlHyIo4QdjnC1YE?pid=ImgGn"
    try:
        response = openai.Image.create(
          prompt=option1,
          n=1,
          size="1024x1024"
        )
        image_url_1 = response['data'][0]['url']
    except Exception as e:
        image_url_1 = defaulturl
    
    try:
        response = openai.Image.create(
          prompt=option2,
          n=1,
          size="1024x1024"
        )
        image_url_2 = response['data'][0]['url']
    except Exception as e:
        image_url_2 = defaulturl
        
    try:
        response = openai.Image.create(
          prompt=option1,
          n=1,
          size="1024x1024"
        )
        image_url_3 = response['data'][0]['url']
    except Exception as e:
        image_url_3 = defaulturl
    
    return [image_url_1, image_url_2, image_url_3]

# Main Game

In [145]:
memory = {'Phase': 'Introduction'}
player_choice = 'None'

for i in range(10):
    phase = ['Choose an Attribute', 'Fight a Creature', 'Evolve to a new species'][i%3]
    res = strict_output(system_prompt = f'''You are the game host for the ”Evolution" game.

The game consists of three phases, which repeats continuously until game over: 
- Choose an attribute: Conditioned on the story, player will choose one attribute to add based on a list of three randomly generated attributes and description of attributes by you. 

- Fight a creature: Player will choose one creature to fight based on a list of three randomly generated creatures by you. Be realistic. 

– Evolve: Player will choose between three species to evolve to, generated by you. Habitat will change based on the new species. This should be the next class in the taxonomy tree, for example, bacteria to protozoa. Once evolved, the species will change but the attributes will remain. Next phase will be choose an attribute.

Introduction: The game starts with a protozoa, and slowly evolves to higher-order species like humans and beyond. The game setting begins in an aquatic pond and starts with the attribute phase with species as Bacteria and attributes as Resilient.''',

user_prompt = f'''Memory: {memory}, Player Choice: {player_choice}, Current Phase: {phase}''',
output_format = {f"What you chose in {memory['Phase']}": f"Option {player_choice}",
                 "Outcome": "Vivid description of outcome",
                 "Species": "Type of Species", "Attribute": "Summarized Attributes", 
                  "Habitat": "Current Habitat", "Phase": f"{phase}",
                  "Option 1": "Option Description",
                  "Option 2": "Option Description",
                  "Option 3": "Option Description"})

    memory = res
    for key in res:
        print(f'{key}: {res[key]}')
        
    image_urls = display_images(res["Option 1"], res["Option 2"], res["Option 3"])
    
    # image_urls = ["https://image.online-convert.com/convert-to-jpg", 
    #               "https://image.online-convert.com/convert-to-jpg", 
    #               "https://image.online-convert.com/convert-to-jpg"]
    
    # HTML and CSS for displaying smaller images side by side
    html = '<div style="display:flex; justify-content:center;">'
    for url in image_urls:
        html += f'<img src="{url}" style="width:200px; height:150px; margin-right: 10px;">'
    html += '</div>'

    # Display the HTML
    display(HTML(html))
        
    player_choice = input('Enter your choice\n')
    while player_choice not in ['1', '2', '3']:
         player_choice = input('Invalid choice. Enter your choice (1, 2 or 3)\n')
    print()

What you chose in Introduction: Option None
Outcome: You are a protozoa in an aquatic pond. As a protozoa, you are a single-celled organism with a simple structure. Your main attribute is being resilient, which allows you to survive in various conditions. However, you lack mobility and are dependent on the surrounding environment for nutrients.
Species: Protozoa
Attribute: Resilient
Habitat: Aquatic Pond
Phase: Choose an Attribute
Option 1: Attribute: Photosynthetic - You gain the ability to produce your own food through photosynthesis, allowing you to survive in areas with abundant sunlight.
Option 2: Attribute: Motile - You gain the ability to move independently, increasing your chances of finding food and escaping predators.
Option 3: Attribute: Filter Feeder - You gain the ability to filter small particles from the water, enabling you to extract nutrients efficiently.


Enter your choice
 2



What you chose in Choose an Attribute: Option 2
Outcome: You have chosen the attribute Motile. As a result, you gain the ability to move independently, increasing your chances of finding food and escaping predators. Your simple structure allows you to move using cilia or flagella, propelling yourself through the water. With this new attribute, you are now more adaptable and have a higher chance of survival in your aquatic pond habitat.
Species: Protozoa
Attribute: Resilient, Motile
Habitat: Aquatic Pond
Phase: Fight a Creature
Option 1: Creature: Amoeba - A single-celled organism similar to you, but lacks the motility attribute. It moves by extending pseudopods and engulfs its food through phagocytosis.
Option 2: Creature: Paramecium - Another single-celled organism with motility attribute. It moves using cilia and feeds on bacteria and other small organisms.
Option 3: Creature: Euglena - A single-celled organism with both motility and photosynthetic attributes. It moves using a whip-

Enter your choice
 3



What you chose in Fight a Creature: Option 3
Outcome: You have chosen to fight the Euglena. As a result of your motility attribute, you are able to swiftly move towards the Euglena, dodging its attempts to escape. With your resilience, you withstand its flagellum strikes and manage to overpower it. You engulf the Euglena, absorbing its nutrients and energy. Your successful hunt has further increased your chances of survival in the aquatic pond habitat.
Species: Protozoa
Attribute: Resilient, Motile
Habitat: Aquatic Pond
Phase: Evolve to a new species
Option 1: Species: Algae - A photosynthetic organism that can produce its own food through photosynthesis. It forms the base of the food chain in the aquatic pond habitat.
Option 2: Species: Amoeba - A single-celled organism similar to you, but lacks the motility attribute. It moves by extending pseudopods and engulfs its food through phagocytosis.
Option 3: Species: Paramecium - Another single-celled organism with motility attribute. It 

KeyboardInterrupt: Interrupted by user